In [ ]:
!pip install geopy

In [ ]:
!pip install tqdm

In [ ]:
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
from tqdm import tqdm
import time

In [ ]:
# obs: trocar o nome do diretório
df = pd.read_csv("C:/Users/jarde/Desktop/git/desafio-poliedro-clusterizacao/dados/microdados_ed_basica_2024.csv", sep =';', encoding='latin1')

In [ ]:

geolocator = Nominatim(user_agent="meu_buscador_de_ceps_123") #agenta para mostraar o progresso


In [ ]:
#cria um dataset auxiliar com as escolas particulares
df_alt = df[df["TP_DEPENDENCIA"] == 4]

In [ ]:
#
df_alt["LOCALIZACAO"] = (
    df_alt["CO_CEP"].astype(str) + ", " +
    df_alt["NO_MUNICIPIO"] + ", " +
    df_alt["NO_UF"] + ", Brasil"
)

In [ ]:
locais_unicos = df_alt["LOCALIZACAO"].dropna().unique()

print(f"Total de linhas: {len(df_alt)}")
print(f"Consultas únicas: {len(locais_unicos)}")


local_to_coords = {}

print("Buscando coordenadas...")
for loc in tqdm(locais_unicos):
    try:
        location = geolocator.geocode(loc)
        if location:
            local_to_coords[loc] = (location.latitude, location.longitude)
        else:
            local_to_coords[loc] = (None, None)

        time.sleep(1)

    except Exception as e:
        print(f"Erro em {loc}: {e}")
        local_to_coords[loc] = (None, None)

df_alt['Latitude'] = df_alt['LOCALIZACAO'].map(lambda x: local_to_coords.get(x, (None, None))[0])
df_alt['Longitude'] = df_alt['LOCALIZACAO'].map(lambda x: local_to_coords.get(x, (None, None))[1])


df_alt.to_csv("ceps_com_coordenadas.csv", index=False)
print("Arquivo salvo: ceps_com_coordenadas.csv")

In [ ]:
caminho_para_salvar = 'C:/Users/jarde/Desktop/git/desafio-poliedro-clusterizacao/dados'  #trocar o nome do diretório
df_alt.to_csv(caminho_para_salvar, index=False, encoding='utf-8-sig')